In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import hvplot.pandas
import plotly.express as px
import scipy.signal as signal
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly.offline import plot, iplot, init_notebook_mode
from plotly.subplots import make_subplots


In [ ]:
def load_csv(name):
    return pd.read_csv(name, header=None, usecols=[3, 4, 10], names=['t', 'vd', 'vt'],float_precision='round_trip')

R = 10

In [ ]:
(-1.8e-4-4e-5)/2

## part 1

In [ ]:
df = load_csv('3.csv')
df['I'] = (df['vt'] - df['vd']) / R 

# df = df[-1.8e-4 < df['t']][df['t'] < -4e-5]
# df = df[-0.00011 < df['t']][df['t'] < -4e-5]
df = df[-1.75e-4 < df['t']][df['t'] < -0.00011]

# df.to_picl('3.pkl')
# df = pd.read_pickle('3.pkl')


plt.figure()
plt.plot(df['t'], df['vd'])
plt.figure()
plt.plot(-df['I'], -df['vd'])

In [ ]:
plt.plot(-df['I'], -df['vd'])
plt.xlabel('I [mA]')
plt.ylabel('vd [V]')
plt.savefig('destination_path.eps', format='eps')

In [ ]:
from scipy.signal import savgol_filter
y = -savgol_filter(df['vd'], 51, 3) # window size 51, polynomial order 3
x = -savgol_filter(df['I'], 51, 3)
plt.plot(x, y)
df2 = pd.DataFrame({'x': x, 'y': y})
df2.to_csv('filtered.csv', index=False)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

# define the exponential function to fit
def func(x, a, b, c):
    return a * np.exp(-b * x) + c


# fit the data to the function
popt, pcov = curve_fit(func, x, y)

# plot the data and fitted curve
plt.plot(x, y, 'o', label='data')
plt.plot(x, func(y, *popt), '-', label='fit')
plt.legend()
plt.show()


In [ ]:
df.hvplot.scatter(x='t', y='vd', width=800, height=400)

In [ ]:
df.hvplot.scatter(x='t', y=['vd', 'vt'], width=800, height=400)

In [ ]:
df = df[-1.8e-4 < df['t']][df['t'] < -4e-5]
df.to_pickle('3.pkl')

In [ ]:
df.hvplot.scatter(x='t', y=['vd', 'vt'], width=800, height=400)
df['I'] = df['vt'] - df['vd'] / R 

In [ ]:
df.hvplot.scatter(x='I', y='vd', width=800, height=400)

## part 2 

In [ ]:
L = 102e-3 # henrry
C = 56.6e-9# farad

In [ ]:
resonance = 1 / np.sqrt(L * C) # 13kHz
zmeasurment_freq = 27.1e3 # 27.1kHz

In [ ]:
df = load_csv('37.4.csv')

In [ ]:
plt.plot(df['t'], df['vd'])

In [ ]:
# df.set_index('t')
n = int(4e5)
sdf = df[n:n+5000]
sdf.reset_index(drop=True, inplace=True)
plt.plot(sdf['vd'])
# px.scatter(sdf, x='t', y='vd')
peaks = signal.find_peaks(-1*sdf['vd'], height=3.4,width = 10, rel_height=1)
plt.plot(peaks[0], -1*peaks[1]['peak_heights'], '.')

In [ ]:
sdf.reset_index(drop=True, inplace=True)
sdf


In [ ]:
peaks = signal.find_peaks(sdf['vd'], height=0.5, distance=1000)
peaks

In [ ]:
trace1 = go.Bar(
    x=sdf['t'],
    y=sdf['vd'],
    # name='points',
    # marker=dict(
    #     color='rgb(255,163,192)'
    #            )
)
trace2 = go.Scatter(
    x=peaks[0],
    y=peaks[1]['peak_heights'],
    name='peaks',
    # yaxis='y2',

)

fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(trace1)
# fig.add_trace(trace2,secondary_y=True)
fig['layout'].update(height = 600, width = 800,xaxis=dict(
      tickangle=-90
    ))
iplot(fig)


In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

trace1 = go.Bar(
    x=df[cat],
    y=df[num],
    name=num,
    marker=dict(
        color='rgb(34,163,192)'
               )
)
trace2 = go.Scatter(
    x=df[cat],
    y=df['cumulative_perc'],
    name='Cumulative Percentage',
    yaxis='y2'

)

fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(trace1)
fig.add_trace(trace2,secondary_y=True)
fig['layout'].update(height = 600, width = 800, title = title,xaxis=dict(
      tickangle=-90
    ))
iplot(fig)


In [ ]:
peaks[0]
peaks[1]['peak_heights']

In [ ]:
import altair as alt
from altair.expr import datum

base = alt.Chart(sdf).encode(
    x='t',
    y='vd',
    color='symbol:N'
).transform_filter(
    datum.symbol == 'GOOG'
)

base.mark_line() + base.mark_point()
